# IBM Data Science Capstone Project

This notebooks contains the data related to the IBM Capstone Project

In [1]:
#data manipulation imports
import pandas as pd
import numpy as np
#data scrapping
from bs4 import BeautifulSoup
import requests

### Scraping data from wikipedia page

In [39]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
# print(soup.prettify())
dataList = []
header = ['Postcode', 'Borough', 'Neighbourhood']

table = soup.find('table',class_='wikitable sortable').tbody

for tableRow in table.findAll('tr'):
    tds = tableRow.find_all('td')
    try:
        #fetch each row and append it to a list
        fetchedRow = Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
        dataList.append(fetchedRow)
    except Exception as e:
        pass
        
df=pd.DataFrame(dataList,columns=header)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [40]:
#set Postcode as the index of dataframe
df.set_index('Postcode')
#select data from the dataframe where Borough is having a value assigned
df = df[df['Borough']!='Not assigned']
#Replace the "/" with a "," in the dataframe. It is a technical requirment
df.replace('/',',', regex = True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [43]:
df.shape

(103, 3)